In [2]:
from constants import *
import dask.dataframe as df
import pandas as pd
import os
from pathlib import Path

rating_csv_path = f'{data_path}/ratings.csv'
rating_csv_path = Path(rating_csv_path).resolve()#.parents[0].absolute

In [2]:
df_rating = df.read_csv(rating_csv_path)
df_rating.dtypes

In [4]:
df_rating.sample(10/len(df_rating)).compute()

/home/asutosh/miniconda3/envs/env_pytorch/lib/python3.8/site-packages/dask/dataframe/core.py:1754: UserWarning: sample does not support the number of sampled items parameter, 'n'. Please use the 'frac' parameter instead.
  warnings.warn(msg)


,userId,movieId,rating,timestamp
989231,6657,724,3.0,884774665
1433017,26486,5500,3.0,1093025484
1777943,44750,115713,4.5,1439846961
249517,51294,1784,4.0,938770276
1055541,72851,22,4.0,949367232
612886,86183,106540,3.5,1413156776
414599,101367,1094,3.0,1146743788
883336,120305,1129,4.0,1225313499
1685573,141777,1639,4.5,1123107954
706838,151072,543,3.5,1107695242


In [5]:
f'number of unique users = {len(df_rating["userId"].unique())}'

'number of unique users = 162541'

In [21]:
df_user_likes = df_rating[df_rating['rating']>=4]
# df_rating['user_likes'] = 
df_user_likes = df_user_likes['movieId'].groupby(df_user_likes['userId']).agg(list).reset_index().compute() # .apply(list, meta=('int64', 'int64')).compute()
df_user_likes

,userId,movieId
0,1,"[296, 307, 665, 1088, 1237, 1250, 1653, 2351, ..."
1,2,"[110, 150, 151, 236, 260, 318, 333, 349, 356, ..."
2,3,"[1, 29, 32, 50, 111, 172, 214, 260, 293, 296, ..."
3,4,"[296, 541, 589, 924, 1036, 1136, 1196, 1197, 1..."
4,5,"[1, 19, 32, 36, 47, 50, 88, 104, 141, 147, 150..."
...,...,...
162337,162537,"[31, 88, 207, 216, 282, 318, 356, 361, 466, 52..."
162338,162538,"[17, 39, 47, 48, 111, 215, 337, 356, 500, 527,..."
162339,162539,"[110, 161, 356, 480, 541, 608, 750, 780, 912, ..."
162340,162540,"[32, 1721, 2205, 3005, 3980, 4167, 4306, 4310,..."


In [22]:
df_user_dislikes = df_rating[df_rating['rating']<4]
# df_rating['user_likes'] = 
df_user_dislikes = df_user_dislikes['movieId'].groupby(df_user_dislikes['userId']).agg(list).reset_index().compute()
df_user_dislikes

,userId,movieId
0,1,"[306, 899, 1175, 1217, 1260, 2011, 2012, 2068,..."
1,2,"[1, 62, 261, 266, 380, 480, 524, 553, 588, 653..."
2,3,"[173, 442, 480, 780, 1127, 1198, 1270, 1320, 1..."
3,4,"[1, 260, 780, 1080, 1200, 1201, 1210, 1214, 12..."
4,5,"[39, 95, 113, 122, 153, 191, 218, 219, 235, 23..."
...,...,...
161506,162537,"[48, 376, 505, 551, 700, 1005, 1556, 1566, 158..."
161507,162538,"[1, 260, 296, 551, 597, 902, 1183, 1230, 1244,..."
161508,162539,"[466, 1748, 4370, 4446]"
161509,162540,"[169, 519, 1125, 1367, 2384, 2986, 3159, 3869,..."


In [29]:
movie_groups = list(df_user_likes['movieId']) + list(df_user_dislikes['movieId'])

In [56]:
from gensim.models import Word2Vec
import datetime

start_time = datetime.datetime.now()
max_window = max([len(group) for group in movie_groups])

model = Word2Vec(movie_groups, epochs=5 , window=max_window,
                 min_count=10, workers=7,
                   sg=1, hs=0, negative=5)

print(f'training time = {datetime.datetime.now()-start_time}')
model.save(f'{output_path}/movie_rec_{datetime.datetime.now()}')


training time = 1:46:38.001172


In [3]:
df_movies = pd.read_csv(f'{data_path}/movies.csv')
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
from gensim.models import Word2Vec

# df_rating[df_rating['movie']]
model = Word2Vec.load(f'{output_path}/movie_rec_2023-05-20 03:12:43.294816')
model.wv.most_similar(1)

[(110, 0.9445347189903259),
 (260, 0.9342778921127319),
 (32, 0.9285299777984619),
 (356, 0.9222207069396973),
 (608, 0.9205285310745239),
 (1198, 0.9202343821525574),
 (480, 0.9188449382781982),
 (1097, 0.9179631471633911),
 (858, 0.9164996147155762),
 (1214, 0.9155781269073486)]